In [1]:
import os
from pydub import AudioSegment
import random
from pydub.playback import play
from pydub.effects import speedup

In [7]:
music_folder = "/Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/music_wav"
speech_folder = "/Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/speech_wav"
output_Mix = "/Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/Mix_wav"
output_Augmentation = "/Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/Augmented_wav"

noise_folder = "/Users/zainhazzouri/projects/Bachelor_Thesis/Data/Noise"

In [6]:
## Create a mix of music and speech

# Make sure output folder exists
os.makedirs(output_Mix, exist_ok=True)

music_files = [f for f in os.listdir(music_folder) if f.endswith('.wav')]
speech_files = [f for f in os.listdir(speech_folder) if f.endswith('.wav')]

for i, (music_file, speech_file) in enumerate(zip(music_files, speech_files)):
    music_path = os.path.join(music_folder, music_file)
    speech_path = os.path.join(speech_folder, speech_file)

    music = AudioSegment.from_wav(music_path)
    speech = AudioSegment.from_wav(speech_path)

    # Mix the two audio files (adjust the volumes if needed)
    mixed = music.overlay(speech)

    # Save the mixed audio to the output folder
    output_path = os.path.join(output_Mix, f'mixed_{i}.wav')
    mixed.export(output_path, format='wav')

print(f'Mixed audio files saved in {output_Mix}')


Mixed audio files saved in /Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/Mix_wav


In [8]:
## preform data augmentation
input_folder = music_folder

# Get a list of all the noise files
noise_files = [f for f in os.listdir(noise_folder) if f.endswith('.wav')]


# Make sure output folder exists
os.makedirs(output_Augmentation, exist_ok=True)

input_files = [f for f in os.listdir(input_folder) if f.endswith('.wav')]

for i, input_file in enumerate(input_files):
    input_path = os.path.join(input_folder, input_file)

    audio = AudioSegment.from_wav(input_path)

    # Change volume
    gain = random.uniform(-10, 10)  # Random gain between -10dB and 10dB
    audio = audio + gain

    # Change pitch
    pitch_factor = random.uniform(0.9, 1.1)  # Random pitch factor between 0.9 and 1.1
    audio = speedup(audio, playback_speed=pitch_factor)

    # Add background noise
    noise_file = random.choice(noise_files)
    noise_path = os.path.join(noise_folder, noise_file)
    noise = AudioSegment.from_wav(noise_path)
    noise = noise - 20  # Reduce noise volume
    audio = audio.overlay(noise)


    # Time stretching
    stretch_factor = random.uniform(0.9, 1.1)  # Random stretch factor between 0.9 and 1.1
    audio = audio.set_frame_rate(int(audio.frame_rate * stretch_factor))

    # Reverse audio
    if random.random() < 0.5:  # 50% chance of reversing audio
        audio = audio.reverse()

    # Save the augmented audio to the output folder
    output_path = os.path.join(output_Augmentation, f'augmented_{i}.wav')
    audio.export(output_path, format='wav')

print(f'Augmented audio files saved in {output_Augmentation}')

Augmented audio files saved in /Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/Augmented_wav


Silence audio file saved at /Users/zainhazzouri/projects/Bachelor_Thesis/Data/Kaggle/silence_wav/silence.wav
